# VOCS data structure 

Variables, Objectives, Constraints, and other Settings (VOCS) helps define our optimization problems. 

In [1]:
from xopt.vocs import VOCS

In [2]:
help(VOCS)

Help on class VOCS in module xopt.vocs:

class VOCS(xopt.pydantic.XoptBaseModel)
 |  VOCS(*, variables: Dict[str, types.ConstrainedListValue] = {}, constraints: Dict[str, types.ConstrainedListValue] = {}, objectives: Dict[str, xopt.vocs.ObjectiveEnum] = {}, constants: Dict[str, Any] = {}, linked_variables: Dict[str, str] = {}) -> None
 |  
 |  Variables, Objectives, Constraints, and other Settings (VOCS) data structure
 |  to describe optimization problems.
 |  
 |  Method resolution order:
 |      VOCS
 |      xopt.pydantic.XoptBaseModel
 |      pydantic.main.BaseModel
 |      pydantic.utils.Representation
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  as_yaml(self)
 |  
 |  constraint_data(self, data: Union[pandas.core.frame.DataFrame, List[Dict]], prefix: str = 'constraint_') -> pandas.core.frame.DataFrame
 |      Returns a dataframe containing constraint data transformed according to
 |      `vocs.constraints` such that values that satisfy each constraint are negat

In [3]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)
vocs

VOCS(variables={'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}, constraints={'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]}, objectives={'c': 'MAXIMIZE', 'd': 'MINIMIZE'}, constants={'g': 1234}, linked_variables={})

In [4]:
# as dict
vocs.dict()

{'variables': {'a': [0.0, 1000.0], 'b': [-1.0, 1.0]},
 'constraints': {'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]},
 'objectives': {'c': 'MAXIMIZE', 'd': 'MINIMIZE'},
 'constants': {'g': 1234},
 'linked_variables': {}}

In [5]:
#  re-parse dict
vocs2 = VOCS.parse_obj(vocs.dict())

In [6]:
# Check that these are the same
vocs2 == vocs

True

In [7]:
# This replaces the old vocs["variables"]
getattr(vocs, "variables")

{'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}

In [8]:
vocs.objectives["c"] == 'MAXIMIZE'

True

In [9]:
# json
vocs.json()

'{"variables": {"a": [0.0, 1000.0], "b": [-1.0, 1.0]}, "constraints": {"e": ["LESS_THAN", 2.0], "f": ["GREATER_THAN", 0.0]}, "objectives": {"c": "MAXIMIZE", "d": "MINIMIZE"}, "constants": {"g": 1234}, "linked_variables": {}}'

# Objective Evaluation

In [10]:
from xopt.vocs import form_objective_data, form_constraint_data, form_feasibility_data
import pandas as pd
import numpy as np

data = pd.DataFrame(vocs.random_inputs(10))
# Add some outputs
data["c"] = data["a"] + data["b"]
data["d"] = data["a"] - data["b"]
data["e"] = data["a"] * 2 + data["b"] * 2
data["f"] = data["a"] * 2 - data["b"] * 2
data.index = np.arange(len(data)) + 5  # custom index
data

,a,b,g,c,d,e,f
5,189.840687,-0.277849,1234,189.562838,190.118536,379.125676,380.237072
6,712.254965,0.281664,1234,712.536629,711.973302,1425.073258,1423.946604
7,631.923627,-0.981855,1234,630.941773,632.905482,1261.883546,1265.810964
8,281.921014,-0.862404,1234,281.058610,282.783418,562.117221,565.566836
9,56.193938,-0.465565,1234,55.728374,56.659503,111.456747,113.319006
10,71.540792,-0.074282,1234,71.466511,71.615074,142.933021,143.230148
11,701.592164,0.855712,1234,702.447877,700.736452,1404.895753,1401.472904
12,497.379692,-0.309039,1234,497.070653,497.688731,994.141305,995.377463
13,951.461424,0.404362,1234,951.865786,951.057062,1903.731572,1902.114124
14,695.550211,-0.568776,1234,694.981436,696.118987,1389.962871,1392.237973


In [11]:
vocs.objectives

{'c': 'MAXIMIZE', 'd': 'MINIMIZE'}

In [12]:
# These are in standard form for minimization
form_objective_data(vocs.objectives, data)

,objective_c,objective_d
5,-189.562838,190.118536
6,-712.536629,711.973302
7,-630.941773,632.905482
8,-281.058610,282.783418
9,-55.728374,56.659503
10,-71.466511,71.615074
11,-702.447877,700.736452
12,-497.070653,497.688731
13,-951.865786,951.057062
14,-694.981436,696.118987


In [13]:
# This is also available as a method
vocs.objective_data(data)

,objective_c,objective_d
5,-189.562838,190.118536
6,-712.536629,711.973302
7,-630.941773,632.905482
8,-281.058610,282.783418
9,-55.728374,56.659503
10,-71.466511,71.615074
11,-702.447877,700.736452
12,-497.070653,497.688731
13,-951.865786,951.057062
14,-694.981436,696.118987


In [14]:
# use the to_numpy() method to convert for low level use.
vocs.objective_data(data).to_numpy()

array([[-189.56283779,  190.11853598],
       [-712.53662889,  711.97330182],
       [-630.94177276,  632.90548193],
       [-281.05861043,  282.78341808],
       [ -55.72837371,   56.65950311],
       [ -71.46651057,   71.61507417],
       [-702.44787665,  700.73645222],
       [-497.07065257,  497.68873133],
       [-951.86578623,  951.05706181],
       [-694.9814356 ,  696.11898675]])

In [15]:
vocs.constraint_data(data)

,constraint_e,constraint_f
5,377.125676,-380.237072
6,1423.073258,-1423.946604
7,1259.883546,-1265.810964
8,560.117221,-565.566836
9,109.456747,-113.319006
10,140.933021,-143.230148
11,1402.895753,-1401.472904
12,992.141305,-995.377463
13,1901.731572,-1902.114124
14,1387.962871,-1392.237973


In [16]:
vocs.feasibility_data(data)

,feasible_e,feasible_f,feasible
5,False,True,False
6,False,True,False
7,False,True,False
8,False,True,False
9,False,True,False
10,False,True,False
11,False,True,False
12,False,True,False
13,False,True,False
14,False,True,False


# Error handling

In [17]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)

In [18]:
d = {'a': [1,2,3]}

df = pd.DataFrame(d)
df2 = pd.DataFrame(df).copy()

df2['b'] = np.nan
df2['b'] - 1

0   NaN
1   NaN
2   NaN
Name: b, dtype: float64

In [19]:
data['a']  = np.nan

In [20]:
a = 2
def f(x=a):
    return x
a=99
f()

2

In [21]:
pd.DataFrame(6e66, index=[1,2,3], columns=['A'])

,A
1,6.000000e+66
2,6.000000e+66
3,6.000000e+66


In [22]:
# These are in standard form for minimization

data = pd.DataFrame({'c':[1,2,3,4]}, index=[9,3,4,5])

form_objective_data(vocs.objectives, data)

,objective_c,objective_d
9,-1.0,inf
3,-2.0,inf
4,-3.0,inf
5,-4.0,inf
